# RunBook - High CPU
Flow
- Find the DB that uses many active connections
- Find the top CPU utilization queries. 
Resources
- https://jfrog.com/community/data-science/troubleshooting-high-cpu-utilization-in-postgresql-databases-a-how-to-guide/ 

One of the patterns of PostgreSQL DB’s leading to high CPU utilization is a high number of active connections.

In [ ]:
sql_connections_overview = """
SELECT * 
FROM ( 
select 
	count(1) as total_connections, 
  sum(case when state!='idle' then 1 else 0 end) as non_idle_connections, 
  sum(case when state='idle' then 1 else 0 end) as idle_connections, 
  round( sum(case when state='idle' then 1 else 0 end)::numeric / count(1), 2) as idle_connections_pct
from pg_stat_activity
) AS t1
CROSS JOIN   
	 (select setting as max_connections 
    from pg_settings 
    where name='max_connections') as s
    """;



Show the number of non-idle connections per DB

In [ ]:
SELECT 
	datname as db_name, 
	count(1) as num_non_idle_connections 
FROM pg_stat_activity 
WHERE state!='idle' 
GROUP BY 1 
ORDER BY num_non_idle_connections DESC;